In [ ]:
import os
import shutil
import paddle
import paddle.dataset.flowers as flowers
import paddle.fluid as fluid
from visualdl import LogWriter
paddle.enable_static()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [ ]:
writer=LogWriter(logdir='work/log/')

In [ ]:
def resnet50(input):
    def conv_bn_layer(input,num_filters,filter_size,stride=1,act=None):
        conv=fluid.layers.conv2d(input=input,
                                num_filters=num_filters,
                                filter_size=filter_size,
                                stride=stride,
                                padding=(filter_size-1)//2,
                                act=None)#因为BN是在激活函数之前的，所以这里用None
        return fluid.layers.batch_norm(input=conv,act=act)
    def shortcut(input,ch_out,stride):
        ch_in=input.shape[1]#NCHW模式，即为通道数
        if ch_in!=ch_out or stride !=1:
            return conv_bn_layer(input,ch_out,1,stride)
        else:
            return input
    def bottleneck_block(input,num_filters,stride):
        conv0=conv_bn_layer(input=input,
                            num_filters=num_filters,
                            filter_size=1,
                            act='relu')
        conv1=conv_bn_layer(input=conv0,
                            num_filters=num_filters,
                            filter_size=3,
                            stride=stride,
                            act='relu')
        conv2=conv_bn_layer(input=conv1,
                            num_filters=num_filters*4,
                            filter_size=1,
                            act=None)
        short=shortcut(input,num_filters*4,stride)
        return fluid.layers.elementwise_add(x=short,y=conv2,act='relu')
    depth=[3,4,6,3]
    num_filters=[64,128,256,512]
    conv=conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2,act='relu')
    conv=fluid.layers.pool2d(input=conv,pool_size=(3,3),pool_stride=(2,2),pool_padding=1,pool_type='max')
    for block in range(len(depth)):
        for i in range(depth[block]):
            conv=bottleneck_block(input=conv,
                                num_filters=num_filters[block],
                                stride=2 if i==0 and block!=0 else 1)
    pool=fluid.layers.pool2d(input=conv,pool_size=7,pool_type='avg',global_pooling=True)
    return pool

image=fluid.data(name='image',shape=[None,3,224,224],dtype='float32')
pool=resnet50(image)
#pool.stop_gradient=True
base_model_program=fluid.default_main_program().clone()

model=fluid.layers.fc(input=pool,size=102,act='softmax')
label=fluid.data(name='label',shape=[None,1],dtype='int64')
cost=fluid.layers.cross_entropy(input=model, label=label)
avg_cost=fluid.layers.mean(cost)#返回值是个一维矩阵
acc=fluid.layers.accuracy(input=model, label=label)

test_program=fluid.default_main_program().clone(for_test=True)

optimizer=fluid.optimizer.AdamOptimizer(learning_rate=1e-3)
opts=optimizer.minimize(avg_cost)

因为cifar10和flower任务相差比较大，原本将resnet50的参数进行了固定，即pool.stop_gradient=True，训练效果较差，因此将其注释#掉了

In [ ]:
train_reader=paddle.batch(flowers.train(),batch_size=32)
test_reader=paddle.batch(flowers.test(),batch_size=32)

In [ ]:
place=fluid.CUDAPlace(0)
exe=fluid.Executor(place)
exe.run(fluid.default_startup_program())
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])

In [ ]:
src_pretrained_model_path='work/pretrained_model/'
def if_exist(var):
    path=os.path.join(src_pretrained_model_path,var.name)
    exist=os.path.exists(path)
    if exist:
        print('load model:%s' % path)
    return exist
fluid.io.load_vars(executor=exe, dirname=src_pretrained_model_path,
                    predicate=if_exist,main_program=base_model_program)

load model:work/pretrained_model/batch_norm_33.w_0
load model:work/pretrained_model/batch_norm_51.w_0
load model:work/pretrained_model/batch_norm_33.w_2
load model:work/pretrained_model/conv2d_33.w_0
load model:work/pretrained_model/conv2d_45.w_0
load model:work/pretrained_model/batch_norm_17.w_1
load model:work/pretrained_model/batch_norm_32.w_2
load model:work/pretrained_model/batch_norm_32.w_0
load model:work/pretrained_model/batch_norm_25.b_0
load model:work/pretrained_model/batch_norm_28.b_0
load model:work/pretrained_model/batch_norm_31.w_2
load model:work/pretrained_model/batch_norm_31.b_0
load model:work/pretrained_model/conv2d_31.w_0
load model:work/pretrained_model/batch_norm_30.w_1
load model:work/pretrained_model/conv2d_1.w_0
load model:work/pretrained_model/conv2d_30.b_0
load model:work/pretrained_model/conv2d_21.b_0
load model:work/pretrained_model/conv2d_30.w_0
load model:work/pretrained_model/batch_norm_43.w_2
load model:work/pretrained_model/batch_norm_35.w_2
load mode

In [ ]:
train_step=0
test_step=0

In [8]:
for pass_id in range(100):
    for batch_id,data in enumerate(train_reader()):
        train_cost, train_acc=exe.run(program=fluid.default_main_program(),
                                    feed=feeder.feed(data),
                                    fetch_list=[avg_cost,acc])
        train_step +=1
        writer.add_scalar(tag="训练/损失值",step=train_step,value=train_cost[0])
        writer.add_scalar(tag="训练/准确率",step=train_step,value=train_acc[0])
        if batch_id %100==0:
            print("Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f" % (pass_id, batch_id,train_cost[0],train_acc[0]))
    test_accs=[]
    test_costs=[]
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc=exe.run(program=test_program,
                                    feed=feeder.feed(data),
                                    fetch_list=[avg_cost,acc])
        test_accs.append(test_acc[0])
        test_costs.append(test_cost[0])
    test_cost=(sum(test_costs)/len(test_costs))
    test_acc=(sum(test_accs)/len(test_accs))
    test_step +=1
    writer.add_scalar(tag="测试/损失值",step=test_step,value=test_cost)
    writer.add_scalar(tag="测试/准确率",step=test_step,value=test_acc)
    print('Test:%d, Cost:%0.5f, Accuracy:%0.5f' % (pass_id, test_cost,test_acc))

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 